### MODEL DESIGNING

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
import cv2 as cv
import os
import PIL
from sklearn.metrics import classification_report
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
pict_size=36
data_dir="./train/"
Type=['FAKE','REAL']
val={'FAKE':0,'REAL':1}
final={0:'FAKE',1:'REAL'}

In [ ]:
X_Train=pd.read_pickle('X.pickle')
Y_Train=pd.read_pickle('Y.pickle')

In [ ]:
model=models.Sequential([
        layers.Conv2D(filters=80,kernel_size=(3,3),padding='same',activation='relu',input_shape=(36,36,3)),
        layers.MaxPool2D((2,2)),
        layers.Dropout((0.2)),
        layers.Conv2D(filters=64,kernel_size=(3,3),padding='same',activation='relu'),
        layers.MaxPool2D((2,2)),
        layers.Dropout((0.2)),
        layers.Conv2D(filters=40,kernel_size=(3,3),padding='same',activation='relu'),
        layers.MaxPool2D((2,2)),
        layers.Dropout((0.1)),
        layers.Conv2D(filters=20,kernel_size=(3,3),padding='same',activation='relu'),
        layers.MaxPool2D((2,2)),
        layers.Dropout((0.1)),
        
        layers.Flatten(),
        layers.Dense(64,activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(48,activation='relu'),
        layers.Dropout(0.2),
        
        layers.Dense(2,activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

###### FILTERS HELP IN EXTRACTING FEATURES FROM THE IMAGES AND THEIR SIZE IS DENOTED BY KERNEL SIZE.
###### STRIDE IS FOR SHIFTING OUR KERNEL BY HOW MANY POSITIONS.
###### MAXPOOLING 2D HELPS IN SIZE REDUCTION USING MAX 
###### PADDING HELPS IN CAPTURING THE FEATURES LOCATED AT EDGES AND CORNERS AND ENFORCES THEIR CONTRIBUTION.


In [ ]:
model.fit(X_Train,Y_Train,epochs=10)

In [ ]:
model.save("Model.h5")

In [ ]:
data_test_dir="./test/"
Testing_data=[]
lis=[]
for categ in Type:
    Files=os.listdir(data_test_dir+categ+"/")
    i=1
    for file in Files:
        imgpath=os.path.join(data_test_dir+categ+"/",file)
        lis.append(imgpath)
        image=cv.imread(imgpath)
        img=cv.imread(str(lis[-1]))
        try:
            new_img=cv.resize(img,(pict_size,pict_size))
            new_img=new_img/255
            Testing_data.append([new_img,val[categ]])
        except:
            print("Not able to preprocess")
        i=i+1

In [ ]:
PIL.Image.open(str(lis[-1]))

In [ ]:
X_Test=[]
Y_Test=[]
for i in range(0,len(Testing_data)):
    X_Test.append(Testing_data[i][0])
    Y_Test.append(Testing_data[i][1])
X_Test=np.array(X_Test)
Y_Test=np.array(Y_Test)
print(X_Test.shape)
print(Y_Test.shape)
y_pred=model.predict(X_Test)

In [ ]:
Y_Pred=[]
for i in range(len(y_pred)):
    y_hat=np.argmax(y_pred[i])
    Y_Pred.append(y_hat)

In [ ]:
print(classification_report(Y_Test,Y_Pred))

In [ ]:

lis=[]
Test=[]
data_test_dir="./Testing/"
Files=os.listdir(data_test_dir)
i=1
for file in Files:
    imgpath=os.path.join(data_test_dir,file)
    lis.append(imgpath)
    image=cv.imread(imgpath)
    img=cv.imread(str(lis[-1]))
    try:
        new_img=cv.resize(img,(36,36))
        new_img=new_img/255
        Test.append(new_img)
    except:
        print("Not able to preprocess")
    print(i)
    i=i+1

In [ ]:
PIL.Image.open(str(lis[-1]))

In [ ]:
y_pred=model.predict(Test)
y_hat=np.argmax(y_pred[0])
print(final[y_hat])